In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
ROOT = ''
TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4ODc3MTlkYS1mMzgzLTQzM2ItYTg5OC1jOGU3MTNmOWZhMjAifQ=='

In [3]:
# !unzip -u '/content/drive/MyDrive/Colab Notebooks/coursework2/faster_rcnn.zip' -d '/content/drive/MyDrive/Colab Notebooks/coursework2/'

In [4]:
import sys
sys.path.insert(0, ROOT)

In [81]:
!pip install einops
!pip install --upgrade --quiet neptune-client
!pip install pytorch-lightning

In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import nn
from RTSD import RTSD
from common import *
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import matplotlib.patches as patches
import tqdm
import neptune.new as neptune
from pytorch_lightning.loggers.neptune import NeptuneLogger
from model50 import *
from model18 import *
from pytorch_lightning.callbacks import ModelCheckpoint
%load_ext autoreload
%autoreload 2


**Training model with resnet50 as backbone and SGD optimizer**

In [4]:
MODEL_NAME = "v1"
PARAMS = {
    'optimizer_params':
      {
        'momentum': 0.9,
        'weight_decay': 0.0005
      },
    'learning_rate': 0.005,
    'num_classes': 198,
    'sys/name': MODEL_NAME,
    'sys/tags': ['resnet50', 'fpn', 'SGD']
}

neptune_logger = NeptuneLogger(
    api_key=TOKEN,
    project_name='sosa/coursework2',
    params=PARAMS,
    upload_source_files=[f'{ROOT}/model50.py', f'{ROOT}/train.ipynb'])

model = LitModel50(
                 optimizer=torch.optim.SGD,
                 learning_rate = PARAMS['learning_rate'],
                 optimizer_params = PARAMS['optimizer_params'],
                 root = ROOT
                )

data_module = RTSD_DataModule(
    batch_size = 8,
    root = ROOT
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=5,
    logger = neptune_logger,
    profiler='simple',

)

NeptuneLogger will work in online mode
                          filename  x_from  y_from  width  height sign_class  \
0  autosave01_02_2012_09_13_33.jpg     649     376     18      18        2_1   
1  autosave01_02_2012_09_13_34.jpg     671     356     20      21        2_1   
2  autosave01_02_2012_09_13_35.jpg     711     332     27      26        2_1   
3  autosave01_02_2012_09_13_36.jpg     764     290     37      36        2_1   
4  autosave01_02_2012_09_13_36.jpg     684     384     17      17       1_23   

   sign_id  
0        0  
1        0  
2        0  
3        0  
4        1  
59188
59188
cnt = 0
53269
5919


MisconfigurationException: GPUs requested but none are available.

In [90]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://app.neptune.ai/sosa/coursework2/e/COUR-26



  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 42.3 M
-------------------------------------
42.1 M    Trainable params
222 K     Non-trainable params
42.3 M    Total params
169.215   Total estimated model params size (MB)


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/threads/ping_thread.py", line 37, in run
    self.__backend.ping_experiment(self.__experiment)
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/api_clients/hosted_api_clients/hosted_alpha_leaderboard_api_client.py", line 391, in ping_experiment
    self.leaderboard_swagger_client.api.ping(experimentId=str(experiment.internal_id)).response().result
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 239, in response
    six.reraise(*sys.exc_info())
  File "/usr/local/lib/python3.7/dist-packages/six.py", line 703, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 124, in wrapper
    return func(self, *args, **

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  8547.2         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8539.7         	|1              	|  8539.7         	|  99.913         	|
run_training_batch                 	|  1.2809         	|6659           	|  8529.4         	|  99.791         	|
optimizer_step_and_closure_0       	|  0.70301        	|6659           	|  4681.4         	|  54.771         	|
training_step_and_backward         	|  0.69866        	|6659           	|  4652.4         	|  54.432         	|
model_forward                      

**Training and testing model with ADAM optimizer**

In [76]:
MODEL_NAME = "v1"
PARAMS = {
    'optimizer_params': {
        'weight_decay': 0.001
    },
    'learning_rate': 0.0001,
    'num_classes': 198,   
}

neptune_logger = NeptuneLogger(
    api_key=TOKEN,
    project_name='sosa/coursework2',
    params=PARAMS,
    name=MODEL_NAME,
    tags=['resnet50', 'fpn', 'ADAM', 'augment'],
    upload_source_files=[f'{ROOT}/model150.py', f'{ROOT}/train.ipynb']
    )

model = LitModel(
                 optimizer=torch.optim.Adam,
                 learning_rate = PARAMS['learning_rate'],
                 optimizer_params = PARAMS['optimizer_params'],
                 root = ROOT
                )

data_module = RTSD_DataModule(
    batch_size = 8,
    root = ROOT
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=5,
    logger = neptune_logger,
    profiler='simple',

)

NeptuneLogger will work in online mode


NameError: ignored

In [ ]:
trainer.fit(model, data_module)

                          filename  x_from  y_from  ...  height  sign_class sign_id
0  autosave01_02_2012_09_13_33.jpg     649     376  ...      18         2_1       0
1  autosave01_02_2012_09_13_34.jpg     671     356  ...      21         2_1       0
2  autosave01_02_2012_09_13_35.jpg     711     332  ...      26         2_1       0
3  autosave01_02_2012_09_13_36.jpg     764     290  ...      36         2_1       0
4  autosave01_02_2012_09_13_36.jpg     684     384  ...      17        1_23       1

[5 rows x 7 columns]
59188
59188
cnt = 0
53269
5919


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://app.neptune.ai/sosa/coursework2/e/COUR-10



  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 42.3 M
-------------------------------------
42.1 M    Trainable params
222 K     Non-trainable params
42.3 M    Total params
169.215   Total estimated model params size (MB)


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/threads/ping_thread.py", line 37, in run
    self.__backend.ping_experiment(self.__experiment)
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/api_clients/hosted_api_clients/hosted_alpha_leaderboard_api_client.py", line 391, in ping_experiment
    self.leaderboard_swagger_client.api.ping(experimentId=str(experiment.internal_id)).response().result
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 239, in response
    six.reraise(*sys.exc_info())
  File "/usr/local/lib/python3.7/dist-packages/six.py", line 703, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 124, in wrapper
    return func(self, *args, **

Saved model!



FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  8646.4         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8608.9         	|1              	|  8608.9         	|  99.565         	|
run_training_batch                 	|  1.2903         	|6659           	|  8592.0         	|  99.37          	|
optimizer_step_and_closure_0       	|  0.99318        	|6659           	|  6613.6         	|  76.489         	|
training_step_and_backward         	|  0.70222        	|6659           	|  4676.1         	|  54.081         	|
model_forward                      

In [ ]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  warnings.warn(*args, **kwargs)


TEST Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
---------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  9171.3         	|  100 %          	|
---------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8608.9         	|1              	|  8608.9         	|  93.868         	|
run_training_batch                 	|  1.2903         	|6659           	|  8592.0         	|  93.683         	|
optimizer_step_and_closure_0       	|  0.99318        	|6659           	|  6613.6         	|  72.112         	|
training_step_and_backward         	|  0.70222        	|6659           	|  4676.1         	|  50.986         	|
model_forward                   


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

**Adam optimizer показал результаты хуже SGD ( не зря во всех статьях используют SGD)**

**Training model with resnet18 as backbone and augment**

In [71]:
checkpoint_callback = ModelCheckpoint(
                                      dirpath=f'{ROOT}/lightning_checkpoints/resnet18/epoch=0-step=6658.ckpt'
                                      )

In [72]:
MODEL_NAME = "v2"
PARAMS = {
    'optimizer_params':
      {
        'momentum': 0.9,
        'weight_decay': 0.0005
      },
    'learning_rate': 0.001,
    'num_classes': 198,
    'sys/name': MODEL_NAME,
    'sys/tags': ['resnet18', 'fpn', 'SGD', 'augment']
}

neptune_logger = NeptuneLogger(
    api_key=TOKEN,
    project_name='sosa/coursework2',
    params=PARAMS,
    upload_source_files=[f'{ROOT}/model18.py', f'{ROOT}/train.ipynb'])

model = LitModel18(
                 optimizer=torch.optim.SGD,
                 learning_rate = PARAMS['learning_rate'],
                 optimizer_params = PARAMS['optimizer_params'],
                 root = ROOT
                )

'''model = LitModel18.load_from_checkpoint(f'{ROOT}/lightning_checkpoints/resnet18/epoch=0-step=6658.ckpt',
                                                         optimizer=torch.optim.SGD,
                                          learning_rate = PARAMS['learning_rate'],
                                          optimizer_params = PARAMS['optimizer_params'],
                                          root = ROOT
                                        )'''


data_module = RTSD_DataModule_Augment(
    batch_size = 8,
    root = ROOT
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=5,
    logger = neptune_logger,
    profiler='simple',
    callbacks=[checkpoint_callback]
)

NeptuneLogger will work in online mode


                          filename  x_from  y_from  ...  height  sign_class sign_id
0  autosave01_02_2012_09_13_33.jpg     649     376  ...      18         2_1       0
1  autosave01_02_2012_09_13_34.jpg     671     356  ...      21         2_1       0
2  autosave01_02_2012_09_13_35.jpg     711     332  ...      26         2_1       0
3  autosave01_02_2012_09_13_36.jpg     764     290  ...      36         2_1       0
4  autosave01_02_2012_09_13_36.jpg     684     384  ...      17        1_23       1

[5 rows x 7 columns]
59188
59188
cnt = 0
53269
5919


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [74]:
test_model(f'{ROOT}/models/model18_epoch0.pt',
           data_module.test_dataloader())

740it [03:02,  4.05it/s]

Final:
Recall = 0.10369932432432426
Precision = 0.16334553303303312


(0.16334553303303312, 0.10369932432432426, 0.1268612440212973)

In [73]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.3 M    Trainable params
0         Non-trainable params
41.3 M    Total params
165.321   Total estimated model params size (MB)


https://app.neptune.ai/sosa/coursework2/e/COUR-24
Saved model!


FileExistsError: ignored